In [2]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

e:\Python\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.14.0-dev20230727


In [7]:
import requests

def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [3]:
# get data files

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# download_url('https://cdn.freecodecamp.org/project-data/sms/train-data.tsv', train_file_path)
# download_url('https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv', test_file_path)

In [44]:
train_dataset = pd.read_table(train_file_path, header=None, names=['label', 'text'])
test_dataset = pd.read_table(test_file_path, header=None, names=['label', 'text'])
test_dataset.head()

,label,text
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [30]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset['text'].values)


In [72]:
model = tf.keras.Sequential([
    encoder,
   tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [75]:
EPOCHS = 10
train_labels = np.asarray([1 if lab == 'spam' else 0 for lab in train_dataset['label'].values])
history = model.fit(train_dataset['text'].values,
                    train_labels,
                    epochs=EPOCHS,
                    validation_steps=30)


Epoch 1/10
131/131 [==============================] - 66s 165ms/step - loss: 0.5890 - accuracy: 0.8626
Epoch 2/10
131/131 [==============================] - 21s 163ms/step - loss: 0.3720 - accuracy: 0.8725
Epoch 3/10
131/131 [==============================] - 20s 156ms/step - loss: 0.1533 - accuracy: 0.9677
Epoch 4/10
131/131 [==============================] - 19s 146ms/step - loss: 0.0845 - accuracy: 0.9828
Epoch 5/10
131/131 [==============================] - 18s 138ms/step - loss: 0.0658 - accuracy: 0.9864
Epoch 6/10
131/131 [==============================] - 18s 135ms/step - loss: 0.0499 - accuracy: 0.9895
Epoch 7/10
131/131 [==============================] - 18s 141ms/step - loss: 0.0430 - accuracy: 0.9911
Epoch 8/10
131/131 [==============================] - 18s 137ms/step - loss: 0.0352 - accuracy: 0.9931
Epoch 9/10
131/131 [==============================] - 20s 149ms/step - loss: 0.0284 - accuracy: 0.9940
Epoch 10/10
131/131 [==============================] - 19s 144ms/step - l

In [76]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = model.predict(np.array([pred_text]))
  prob = prediction[0][0]
  p_label = 'ham'
  if prob > 0.5:
    p_label='spam'
  return (prob, p_label)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 11s 11s/step
(0.00031913968, 'ham')


In [78]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 39ms/step
You passed the challenge. Great job!
